## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the acute nephritis dataset loaded below and analyze the model generated for this dataset.

In [0]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
# columns: 
# Temperature of patient { 35C-42C }
# Occurrence of nausea { yes, no }
# Lumbar pain { yes, no }
# Urine pushing (continuous need for urination) { yes, no }
# Micturition pains { yes, no }
# Burning of urethra, itch, swelling of urethra outlet { yes, no }
# decision: Nephritis of renal pelvis origin { yes, no } 

cols = ['temp', 'nausea', 'lumbar_pain', 'urine_pushing', 'micturition_pains', 'burning', 'nephritis']
nephritis = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/acute.csv', names=cols)

Recall that we need to check for missing data and create dummy variables from the non-numeric columns. Perform both steps below:

In [5]:
# answer below:
print(nephritis.isna().sum().sum())
dummies=pd.get_dummies(nephritis, drop_first=True)

0


In [6]:
dummies.head()

,temp,nausea_yes,lumbar_pain_yes,urine_pushing_yes,micturition_pains_yes,burning_yes,nephritis_yes
35,5,0,1,0,0,0,0
35,9,0,0,1,1,1,0
35,9,0,1,0,0,0,0
36,0,0,0,1,1,1,0
36,0,0,1,0,0,0,0


Scale only the independent variables using the minmax scaler.

In [53]:
# answer below:
X= dummies.drop('nephritis_yes', axis =1)
y= dummies['nephritis_yes']
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#X = MinMaxScaler().fit_transform(X)
type(X)

pandas.core.frame.DataFrame

Create a train test split with 20% of the data in the test subsample.

In [0]:
# answer below:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#X_train=MinMaxScaler().fit_transform(X_train)
#X_test=MinMaxScaler().fit_transform(X_test)

Create a KNN model for our scaled data with k=5 and report the accuracy score.

In [85]:
# answer below:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors= 25)
results = classifier.fit(X_train, y_train)
print('Train score: ',results.score(X_train,y_train))
print('Test score:', results.score(X_test,y_test))

Train score:  0.7291666666666666
Test score: 0.4583333333333333


When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [86]:
# answer below:

classifier = KNeighborsClassifier(n_neighbors= 25, weights='distance')
results = classifier.fit(X_train, y_train)
print('Train score: ',results.score(X_train,y_train))
print('Test score:', results.score(X_test,y_test))

Train score:  1.0
Test score: 0.8333333333333334


In [0]:
import math
def _get_weights(dist):
    """Get the weights from an array of distances and a parameter ``weights``
    Parameters
    ----------
    dist : ndarray
        The input distances
    weights : {'uniform', 'distance' or a callable}
        The kind of weighting used
    Returns
    -------
    weights_arr : array of the same shape as ``dist``
        if ``weights == 'uniform'``, then returns None
    """
    if dist.dtype is np.dtype(object):
        for point_dist_i, point_dist in enumerate(dist):
                # check if point_dist is iterable
                # (ex: RadiusNeighborClassifier.predict may set an element of
                # dist to 1e-6 to represent an 'outlier')
            if hasattr(point_dist, '__contains__') and 0. in point_dist:
                dist[point_dist_i] = point_dist == 0.
            else:
                dist[point_dist_i] = 1. / point_dist**0.5
    else:
        with np.errstate(divide='ignore'):
            dist = 1. / dist**6
            inf_mask = np.isinf(dist)
            inf_row = np.any(inf_mask, axis=1)
            dist[inf_row] = inf_mask[inf_row]
    return dist
        
    

In [106]:
classifier = KNeighborsClassifier(n_neighbors= 25, weights= _get_weights)
results = classifier.fit(X_train, y_train)
print('Train score: ',results.score(X_train,y_train))
print('Test score:', results.score(X_test,y_test))

Train score:  1.0
Test score: 0.9583333333333334
